# Import Libraries

In [117]:
%matplotlib inline
from tqdm import tqdm
import sys
import statistics
import datacube
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
sys.path.insert(1, "../Tools/")
from dea_tools.plotting import rgb, display_map
from dea_tools.landcover import plot_land_cover
from matplotlib import colors as mcolours
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

# Create a datacube

In [2]:
dc = datacube.Datacube(app="DEA_Land_Cover_Savannah_2")

# Choose a product

In [3]:
product = "ga_ls_landcover_class_cyear_2"

measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
level3,level3,uint8,1,0,NaN,NaN
lifeform_veg_cat_l4a,lifeform_veg_cat_l4a,uint8,1,0,[lifeform],NaN
canopyco_veg_cat_l4d,canopyco_veg_cat_l4d,uint8,1,0,[vegetation_cover],NaN
watersea_veg_cat_l4a_au,watersea_veg_cat_l4a_au,uint8,1,0,[water_seasonality],NaN
waterstt_wat_cat_l4a,waterstt_wat_cat_l4a,uint8,1,0,[water_state],NaN
inttidal_wat_cat_l4a,inttidal_wat_cat_l4a,uint8,1,0,[intertidal],NaN
waterper_wat_cat_l4d_au,waterper_wat_cat_l4d_au,uint8,1,0,[water_persistence],NaN
baregrad_phy_cat_l4d_au,baregrad_phy_cat_l4d_au,uint8,1,0,[bare_gradation],NaN
level4,level4,uint8,1,0,[full_classification],NaN


# Load Geojson file for Savannah 

In [4]:
# Load the GeoJSON file
shapefile = gpd.read_file('NAust_mask_IBRA_WGS1984.geojson')

# Print the shapefile
print(shapefile)

   FID REG_CODE_7    REG_NAME_7     HECTARES       SQ_KM  REC_ID REG_CODE_6  \
0    0        ARC  Arnhem Coast  3335668.565  110.782098       1        ARC   

     REG_NAME_6  REG_NO_61          FEAT_ID  Shape_Leng  Shape_Area  \
0  Arnhem Coast         81  GA_100K_Islands   52.135362    2.774143   

                                            geometry  
0  POLYGON ((132.88061 -11.33308, 132.87942 -11.3...  


In [5]:
geojson_dict = shapefile.geometry.__geo_interface__

In [36]:
# Get the bounding box of the GeoJSON
bounds = shapefile.total_bounds

In [105]:
# Get the width and height of the bounding box
width = bounds[2] - bounds[0]
height = bounds[3] - bounds[1]

In [107]:
# Define the desired chunk width and height
chunk_width = 0.05 # in degrees
chunk_height = 0.05 # in degrees

# Calculate the number of chunks needed to cover the entire area
# bounds is a tuple containing (left, bottom, right, top) coordinates
n_chunks_x = int(np.ceil((bounds[2] - bounds[0]) / chunk_width))  # number of chunks in the x direction
n_chunks_y = int(np.ceil((bounds[3] - bounds[1]) / chunk_height))  # number of chunks in the y direction
n_chunks = n_chunks_x * n_chunks_y  # total number of chunks needed to cover the area
print(n_chunks)

# Create a list to store the chunks
chunks = []

# Loop through all the chunks and create a Polygon for each one
for i in range(n_chunks_y):  # loop through the y direction
    for j in range(n_chunks_x):  # loop through the x direction
        # Calculate the bounds of the current chunk
        left = bounds[0] + j * chunk_width
        right = bounds[0] + (j + 1) * chunk_width
        bottom = bounds[1] + i * chunk_height
        top = bounds[1] + (i + 1) * chunk_height

        # Create a Polygon object for the current chunk
        polygon = Polygon([(left, bottom), (left, top), (right, top), (right, bottom)])

        # Add the Polygon to the list of chunks
        chunks.append(MultiPolygon([polygon]))

125307


In [63]:
# Create a list of chunks
chunks = []
for i in range(n_chunks):
    # Calculate the bounds of the chunk
    left = bounds[0] + i * chunk_width
    right = bounds[0] + (i + 1) * chunk_width
    bottom = bounds[1]
    top = bounds[1] + chunk_height

    # Create the polygon for the chunk
    polygon = Polygon([(left, bottom), (left, top), (right, top), (right, bottom)])
    
    # Add the chunk to the list
    chunks.append(MultiPolygon([polygon]))

# # Print the chunks
# print(chunks)

In [108]:
# Define a dictionary of properties for each chunk
properties = {"name": [f"chunk_{i}" for i in range(n_chunks)]}

In [109]:
# Create a GeoDataFrame for the chunks
chunks_gdf = gpd.GeoDataFrame(properties, geometry=chunks)

# Print the GeoDataFrame
print(chunks_gdf)

                name                                           geometry
0            chunk_0  MULTIPOLYGON (((119.36611 -21.71351, 119.36611...
1            chunk_1  MULTIPOLYGON (((119.41611 -21.71351, 119.41611...
2            chunk_2  MULTIPOLYGON (((119.46611 -21.71351, 119.46611...
3            chunk_3  MULTIPOLYGON (((119.51611 -21.71351, 119.51611...
4            chunk_4  MULTIPOLYGON (((119.56611 -21.71351, 119.56611...
...              ...                                                ...
125302  chunk_125302  MULTIPOLYGON (((147.46611 -10.71351, 147.46611...
125303  chunk_125303  MULTIPOLYGON (((147.51611 -10.71351, 147.51611...
125304  chunk_125304  MULTIPOLYGON (((147.56611 -10.71351, 147.56611...
125305  chunk_125305  MULTIPOLYGON (((147.61611 -10.71351, 147.61611...
125306  chunk_125306  MULTIPOLYGON (((147.66611 -10.71351, 147.66611...

[125307 rows x 2 columns]


In [ ]:
savannah_chips_sampled=[]
for chunk in tqdm(random.sample(chunks_gdf.to_dict("records"),200)):
    # Create the 'query' dictionary object, which contains the longitudes, latitudes and time defined above
    query = {
        "y": (chunk['geometry'].bounds[1],chunk['geometry'].bounds[3]),
        "x": (chunk['geometry'].bounds[0],chunk['geometry'].bounds[2]),
        "time": (1988,2020),
    }

    # Load DEA Land Cover data from the datacube
    chip = dc.load(
        product="ga_ls_landcover_class_cyear_2",
        output_crs="EPSG:3577",
        resolution=(-25, 25),
        **query
    )
    savannah_chips_sampled.append(chip)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
chunk_0=chunks_gdf.to_dict('records')[70]

In [112]:
display_map(x=(chunk_0['geometry'].bounds[0],chunk_0['geometry'].bounds[2]), y=(chunk_0['geometry'].bounds[1],chunk_0['geometry'].bounds[3]))